# Lab 5 : Mixture Models+Model order selection 

Welcome to the advanced Machine Learning Course.

The objective of this lab session is to code a few regression algorithms and to apply them to synthetic and real datasets.

Please put **"ML - MDS - TD5"** in the mail subject or I might lose your work (which means 0) and send it to pierre.houdouin@centralesupelec.fr

Please label your notebook **"L5_familyname1_familyname2.ipynb"** or I might lose your work (which means 0).

We begin with the standard imports:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import norm
import warnings
import random
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd


from matplotlib.patches import Ellipse



warnings.filterwarnings('ignore')

%matplotlib inline
plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}

## GMM

A Gaussian mixture model is a probabilistic model that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. After estimation of those parameters we get an estimation of the distribution of our data. For the clustering task, one can think of mixture models as generalizing k-means clustering to incorporate information about the covariance structure of the data as well as the centers of the latent Gaussians. 

### First part

Fill in the following class to implement a multivariate GMM:


### Initialization:

1. Initialize the means $\mu^{(1)}, \ldots, \mu^{(k)}$, covariances $\Sigma^{(1)}, \ldots, \Sigma^{(k)}$, and mixing coefficients $\alpha^{(1)}, \ldots, \alpha^{(k)}$, where $k$ is the number of components in the mixture model.

### Expectation Step (E-step):

2. Calculate the responsibilities $P_{n,k}$ that estimate the probability that data point $x_n$ is generated by component $k$:

$$
P_{n,k} = \frac{\alpha^{(k)} \mathcal{N}(x_n | \mu^{(k)}, \Sigma^{(k)})}{\sum_{j=1}^{K} \alpha^{(j)} \mathcal{N}(x_n | \mu^{(j)}, \Sigma^{(j)})}
$$

### Maximization Step (M-step):
We are looking to minimize by adding the constraint $\sum_{k=1}^{K} \pi_k = 1$ and noting $\theta = (\pi_1,..., \pi_K, \mu_1,..., \mu_K, \Sigma_1,..., \Sigma_K, \lambda)$, to minimize using the Lagrangian method:

$$
L_r(\theta) = \sum_{i=1}^{n} \sum_{k=1}^{K} P_{i,k} \left[ \log(\pi_k) - \frac{1}{2} \log(2\pi) - \frac{1}{2} \log(|\Sigma|) - \frac{1}{2} (x_i - \mu_k)^T \Sigma^{-1} (x_i - \mu_k) \right] - \lambda \left( \sum_{k=1}^{K} \pi_k - 1 \right)
$$

We will therefore seek to cancel the partial derivatives with respect to all components of $\theta$. Let's start with the $\pi_k$ and $\lambda$:

For $\forall k \in [1, K]$,

$$
\frac{\partial L_r(\theta)}{\partial \pi_k} = \sum_{i=1}^{n} \frac{P_{i,k}}{\pi_k} - \lambda = 0 \Rightarrow \lambda = \sum_{i=1}^{n} P_{i,k}
$$

Then,

$$
\frac{\partial L_r(\theta)}{\partial \lambda} = -\left( \sum_{k=1}^{K} \pi_k - 1 \right) = 0 \Rightarrow \sum_{k=1}^{K} \pi_k = 1
$$

From which it comes:

$$
1 = \sum_{k=1}^{K} \sum_{i=1}^{n} P_{i,k} \Rightarrow \sum_{i=1}^{n} \sum_{k=1}^{K} P_{i,k} = n
$$

Thus, for $\forall k \in [1, K]$,

$$
\pi_k = \frac{1}{n} \sum_{i=1}^{n} P_{i,k}
$$




∀k ∈ [1, K],

\begin{align*}
\frac{\partial L(\theta)}{\partial μ_k} &= \sum_{i=1}^n p_{i_k} \frac{\partial}{\partial μ_k} \left( - \frac{1}{2} (x_i - μ_k)^T Σ_k^{-1} (x_i - μ_k) \right) \\
&= \sum_{i=1}^n \frac{p_{i_k}}{2} (x_i - μ_k)^T (\Sigma_k^{-1} + (\Sigma_k^{-1})^T) \\
&= \sum_{i=1}^n p_{i_k} (x_i - μ_k)^T Σ_k^{-1} \\
&= \sum_{i=1}^n p_{i_k} Σ_k^{-1}(x_i - μ_k) \\
&= Σ_k^{-1} \sum_{i=1}^n p_{i_k} (x_i - μ_k)
\end{align*}

car $Σ_k$ est symétrique donc $Σ_k^{-1}$ aussi. On annule cette dérivée partielle et il vient

$$\frac{\partial L(\theta)}{\partial μ_k} = 0 ⟹ μ_k = \frac{1}{nπ_k} \sum_{i=1}^n p_{i_k} x_i ⟹ μ_k = \frac{1}{nπ_k} \sum_{i=1}^n p_{i_k} x_i$$



Given:

$$ \frac{\partial L_r(\theta)}{\partial \mu_k} = 0 \Rightarrow \mu_k = \frac{1}{n_k} \sum_{i=1}^{n} P_{i,k} x_i $$


$$ \left| \Sigma_k^{-1} \right| = (\left| \Sigma_k \right|)^{-1} \Rightarrow \log(\left| \Sigma_k \right|) = -\log(\left| \Sigma_k^{-1} \right|) \Rightarrow \frac{1}{2} \log(\left| \Sigma_k \right|) = -\frac{1}{2} \log(\left| \Sigma_k^{-1} \right|) $$

or, si $\Sigma$ est une matrice inversible,

$$ \frac{\partial \log(|\Sigma|)}{\partial \Sigma} = \Sigma^{-1} \Rightarrow \frac{\partial}{\partial \Sigma_k} (-\frac{1}{2} \log(\left| \Sigma_k \right|)) = \frac{1}{2} \Sigma_k^{-1} $$

Par ailleurs,

$$\frac{\partial a^T X b}{\partial X} = ba^T \Rightarrow \frac{\partial}{\partial \Sigma_k} ((x_i - \mu_k)^T \Sigma_k^{-1} (x_i - \mu_k)) = (x_i - \mu_k)(x_i - \mu_k)^T $$

Ce qui permet de trouver:

$ \forall k \in [1, K], $

$$ \frac{\partial L_r(\theta)}{\partial \Sigma^{-1}_k} = \sum_{i=1}^{n} P_{i,k} \left[ \frac{1}{2} \Sigma_k - \frac{1}{2} (x_i - \mu_k)(x_i - \mu_k)^T \right] $$

Ce qui permet de trouver, en annulant la dérivée partielle:

$ \forall k \in [1, K], $

$$ \frac{\partial L_r(\theta)}{\partial \Sigma^{-1}_k} = 0 \Rightarrow \Sigma_k = \frac{1}{n_k} \sum_{i=1}^{n} P_{i,k} (x_i - \mu_k)(x_i - \mu_k)^T $$



In [2]:
class my_GMM():
    
    def __init__(self, k):
        '''
        Parameters:
        k: integer
            number of components
        
        Attributes:
        alpha_: np.array
            proportion of components
        mu_: np.array
            array containing means
        Sigma_: np.array
            array cointaining covariance matrix
        cond_prob_: (n, K) np.array
            conditional probabilities for all data points 
        labels_: (n, ) np.array
            labels for data points
        '''
        self.k = k
        self.alpha_ = None
        self.mu_ = None
        self.Sigma_ = None
        self.cond_prob_ = None
        self.labels_ = None


        
    def fit(self, X):
        """ Find the parameters
        that better fit the data
        
        Parameters:
        -----------
        X: (n, p) np.array
            Data matrix
        
        Returns:
        -----
        self
        """
        
        n, p = X.shape

        # Initialize parameters using K-Means
        kmeans = KMeans(n_clusters=self.k, random_state=0).fit(X)
        self.labels_ = kmeans.labels_
        self.alpha_ = np.array([np.mean(self.labels_ == i) for i in range(self.k)])
        self.mu_ = np.array([np.mean(X[self.labels_ == i], axis=0) for i in range(self.k)])
        self.Sigma_ = np.array([np.cov(X[self.labels_ == i], rowvar=False) for i in range(self.k)])
        
        # EM Algorithm
        num_iterations = 100
        for _ in range(num_iterations):
            # Compute conditional probability matrix Step E
            self.cond_prob_ = self.compute_condition_prob_matrix(X, self.alpha_, self.mu_, self.Sigma_)
            
            # Update parameters step M
            Nk = np.sum(self.cond_prob_, axis=0)
            self.alpha_ = Nk / n
            self.mu_ = np.dot(self.cond_prob_.T, X) / Nk[:, np.newaxis]
            self.Sigma_ = [np.dot(self.cond_prob_[:, i] * (X - self.mu_[i]).T, X - self.mu_[i]) / Nk[i] for i in range(self.k)]

        return self
        
    def predict(self, X):
        """ Predict labels for X
        
        Parameters:
        -----------
        X: (n, p) np.array
            New data matrix
        
        Returns:
        -----
        label assignment        
        """
        proba = self.compute_proba(X)
        self.labels_ = np.argmax(proba, axis=1)
        return self.labels_
        
    def compute_proba(self, X):
        """ Compute probability vector for X
        
        Parameters:
        -----------
        X: (n, p) np.array
            New data matrix
        
        Returns:
        -----
        proba: (n, k) np.array        
        """
        proba = self.compute_condition_prob_matrix(X, self.alpha_, self.mu_, self.Sigma_)
        return proba
    
    def compute_condition_prob_matrix(self, X, alpha, mu, Sigma):
        '''Compute the conditional probability matrix 
        shape: (n, K)
        '''
        k = self.k
        n, p = X.shape
        prob_matrix = np.zeros((n, k))
        
        for i in range(k):
            prob_matrix[:, i] = alpha[i] * multivariate_normal.pdf(X, mean=mu[i], cov=Sigma[i], allow_singular=True)
            
        prob_matrix /= prob_matrix.sum(axis=1)[:, np.newaxis]
        
        return prob_matrix
    
    




# Test of the GMM algorithm

Generate your own mixture of Gaussian distributions to test the model, choose parameters so that GMM performs better than K-Means on it. Use `np.random.multivariate_normal`. 

Plot data with colors representing predicted labels and shapes representing real labels.

In [3]:
n_samples = 300
#On définit des paramètres
mean1 = [2, 3]
mean2 = [7, 7]
mean3 = [10, 2]
cov1 = [[1, -0.5], [-0.5, 1]]
cov2 = [[1, 0], [0, 1]]
cov3 = [[1, 0], [0, 1]]

# On génère des données aléatoires
data1 = np.random.multivariate_normal(mean1, cov1, n_samples // 3)
data2 = np.random.multivariate_normal(mean2, cov2, n_samples // 3)
data3 = np.random.multivariate_normal(mean3, cov3, n_samples // 3)



df1 = pd.DataFrame({'X': data1[:, 0], 'Y': data1[:, 1], 'Dataset': 'Data 1'})
df2 = pd.DataFrame({'X': data2[:, 0], 'Y': data2[:, 1], 'Dataset': 'Data 2'})
df3 = pd.DataFrame({'X': data3[:, 0], 'Y': data3[:, 1], 'Dataset': 'Data 3'})
df = pd.concat([df1, df2, df3])


# Analyse graphique
fig = px.scatter(df, x='X', y='Y', color='Dataset', marginal_x='histogram', marginal_y='histogram')

fig.update_xaxes(showgrid=False, zeroline=False) 
fig.update_yaxes(showgrid=False, zeroline=False)  

fig.update_traces(marker=dict(size=4), selector=dict(mode='markers+text'))
fig.update_traces(marker=dict(opacity=0.7), selector=dict(type='scatter'))
fig.update_traces(marker=dict(line=dict(width=0)), selector=dict(type='scatter'))

fig.update_layout(template="plotly_dark")  

# Afficher le graphique
fig.show()



In [4]:
np.random.seed(0)

n_features = 2
n_clusters = 3

X = np.vstack((data1, data2, data3))

# K-Means
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
kmeans_labels = kmeans.labels_

# GMM
gmm = my_GMM(n_clusters)
gmm.fit(X)
gmm_labels = gmm.predict(X)

# Affichage graphique & comparaison
fig1 = px.scatter(x=X[:, 0], y=X[:, 1], color=kmeans_labels, title='K-Means Clustering', labels={'color': 'K-Means Labels'})
fig1.update_traces(marker=dict(symbol='circle'))
fig1.update_xaxes(title_text='Feature 1', showgrid=False, zeroline=False)  # Désactivez la grille et la barre en X
fig1.update_yaxes(title_text='Feature 2', showgrid=False, zeroline=False)  # Désactivez la grille et la barre en Y
fig1.update_layout(width=800)
fig1.update_layout(template="plotly_dark")  

fig2 = px.scatter(x=X[:, 0], y=X[:, 1], color=gmm_labels, title='GMM Clustering', labels={'color': 'GMM Labels'})
fig2.update_traces(marker=dict(symbol='square'))
fig2.update_xaxes(title_text='Feature 1', showgrid=False, zeroline=False)  # Désactivez la grille et la barre en X
fig2.update_yaxes(title_text='Feature 2', showgrid=False, zeroline=False)  # Désactivez la grille et la barre en Y
fig2.update_layout(width=800)
fig2.update_layout(template="plotly_dark")

fig1.show()
fig2.show()

without any surprise, with very different center of our distributions, the performance of the GMM and the KNN algorithm is precisly the same.  

To better illustrate the work of the GMM, we can draw the density of the best parameters: 

In [5]:
def plot_gmm_densities_2d(gmm):
    fig = go.Figure()

    for i in range(gmm.k):
        mu = gmm.mu_[i]
        Sigma = gmm.Sigma_[i]

        fig.add_trace(go.Scatter(x=[mu[0]], y=[mu[1]], mode='markers', name=f'Component {i + 1}', marker=dict(size=8, symbol='x', color='red')))

        eigvalues, eigvectors = np.linalg.eigh(Sigma)
        width, height = 2 * np.sqrt(2) * np.sqrt(eigvalues)

        t = np.linspace(0, 2 * np.pi, 100)
        x = mu[0] + width * np.cos(t)
        y = mu[1] + height * np.sin(t)

        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='blue', width=2)))

    fig.update_layout(xaxis_title='Feature 1')
    fig.update_layout(yaxis_title='Feature 2')
    fig.update_layout(title='GMM Component Densities in 2D')
    fig.update_layout(template="plotly_dark")

    fig.show()

plot_gmm_densities_2d(gmm)


Let's challenge our algorithm and try this in an upper scale

In [6]:

# Generate synthetic data
np.random.seed(0)
n_samples = 300
n_features = 3  # changed from 2 to 3
n_clusters = 3

# Create three random Gaussian distributions
mean1 = np.array([2, 3, 4])
cov1 = np.array([[1, -0.5, 0.5], [-0.5, 1, 0.5], [0.5, 0.5, 1]])
data1 = np.random.multivariate_normal(mean1, cov1, n_samples // 3)

mean2 = np.array([7, 7, 6])
cov2 = np.array([[1, 0, 0.5], [0, 1, -0.5], [0.5, -0.5, 1]])
data2 = np.random.multivariate_normal(mean2, cov2, n_samples // 3)

mean3 = np.array([10, 2, 3])
cov3 = np.array([[1, 0.5, 0], [0.5, 1, -0.5], [0, -0.5, 1]])
data3 = np.random.multivariate_normal(mean3, cov3, n_samples // 3)

X = np.vstack((data1, data2, data3))

# Fit K-Means
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
kmeans_labels = kmeans.labels_

# Fit your GMM
gmm = my_GMM(n_clusters)
gmm.fit(X)
gmm_labels = gmm.predict(X)

# Create 3D plots using plotly
fig = go.Figure()

# K-Means Clustering
fig.add_trace(go.Scatter3d(x=X[:, 0], y=X[:, 1], z=X[:, 2], mode='markers', 
                           marker=dict(color=kmeans_labels, size=5, opacity=0.8, colorscale='viridis'),
                           name='K-Means'))

# GMM Clustering
fig.add_trace(go.Scatter3d(x=X[:, 0], y=X[:, 1], z=X[:, 2], mode='markers', 
                           marker=dict(color=gmm_labels, size=5, opacity=0.8, symbol='square', colorscale='viridis'),
                           name='GMM'))

fig.update_layout(title="3D Clustering (K-Means and GMM)", scene=dict(
        xaxis=dict(title='Feature 1'),
        yaxis=dict(title='Feature 2'),
        zaxis=dict(title='Feature 3')))

# Appliquez le modèle de thème sombre
fig.update_layout(template="plotly_dark")

# Affichez le graphique
fig.show()


Even in greater dimensions, the GMM performance is the same as KMEANS for this simple dataset.   
Let's try with a different dataset with more confused centers

In [7]:
# Generate synthetic data
np.random.seed(0)
n_samples = 300
n_features = 3  
n_clusters = 3

# Generate synthetic data
np.random.seed(0)

# Create three random Gaussian distributions in 3D that cross each other

mean1 = np.array([4, 5, 4])
cov1 = np.array([[2, -0.5, 1], [-0.5, 2, 1], [1, 1, 2]])  # increased variance
data1 = np.random.multivariate_normal(mean1, cov1, n_samples // 3)

mean2 = np.array([6, 6, 5])
cov2 = np.array([[2, 0.5, 1], [0.5, 2, -1], [1, -1, 2]])  # increased variance
data2 = np.random.multivariate_normal(mean2, cov2, n_samples // 3)

mean3 = np.array([8, 4, 5])
cov3 = np.array([[2, 0.5, 0.5], [0.5, 2, -1], [0.5, -1, 2]])  # increased variance
data3 = np.random.multivariate_normal(mean3, cov3, n_samples // 3)

X = np.vstack((data1, data2, data3))


# Fit K-Means
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
kmeans_labels = kmeans.labels_

# Fit your GMM
gmm = my_GMM(n_clusters)
gmm.fit(X)
gmm_labels = gmm.predict(X)

# Create 3D plots using plotly
fig = go.Figure()

# K-Means Clustering
fig.add_trace(go.Scatter3d(x=X[:, 0], y=X[:, 1], z=X[:, 2], mode='markers', 
                           marker=dict(color=kmeans_labels, size=5, opacity=0.8, colorscale='viridis'),
                           name='K-Means'))

# GMM Clustering
fig.add_trace(go.Scatter3d(x=X[:, 0], y=X[:, 1], z=X[:, 2], mode='markers', 
                           marker=dict(color=gmm_labels, size=5, opacity=0.8, symbol='square', colorscale='viridis'),
                           name='GMM'))

fig.update_layout(title="3D Clustering (K-Means and GMM)", scene=dict(
        xaxis=dict(title='Feature 1'),
        yaxis=dict(title='Feature 2'),
        zaxis=dict(title='Feature 3')))

# Appliquez le modèle de thème sombre
fig.update_layout(template="plotly_dark")

# Affichez le graphique
fig.show()


In this example, it's crystal clear that the GMM is more efficient than the KMEANS.   
When the label are merged, our GMM model is more capable to find some good clusters, where the KMEANS finds more disparate ones  
Let's explore the densities determined in this example 

In [8]:
def plot_gmm_densities_3d(gmm):
    fig = go.Figure()

    for i in range(gmm.k):
        mu = gmm.mu_[i]
        Sigma = gmm.Sigma_[i]

        # Points for the mean (mu)
        fig.add_trace(go.Scatter3d(x=[mu[0]], y=[mu[1]], z=[mu[2]], mode='markers', name=f'Component {i + 1}', marker=dict(size=5, symbol='x', color='red')))

        # Compute the eigenvectors and eigenvalues for the covariance (Sigma)
        eigvalues, eigvectors = np.linalg.eigh(Sigma)
        width, height, depth = 2 * np.sqrt(2) * np.sqrt(eigvalues)

        # Create 3D ellipsoid to represent the covariance
        u = np.linspace(0, 2 * np.pi, 100)
        v = np.linspace(0, np.pi, 100)
        x = width * np.outer(np.cos(u), np.sin(v)) + mu[0]
        y = height * np.outer(np.sin(u), np.sin(v)) + mu[1]
        z = depth * np.outer(np.ones_like(u), np.cos(v)) + mu[2]

        fig.add_trace(go.Surface(x=x, y=y, z=z, colorscale='blues', opacity=0.2))

    fig.update_layout(scene=dict(
        xaxis_title='Feature 1',
        yaxis_title='Feature 2',
        zaxis_title='Feature 3',
    ))

    fig.update_layout(title='GMM Component Densities in 3D')
    fig.update_layout(template="plotly_dark")
    fig.show()

# Utilisez cette fonction pour afficher les densités de composantes GMM en 3D avec Plotly
plot_gmm_densities_3d(gmm)


To conclude, we develop an GMM algorithm more efficient thant the K MEANS, and propose many ways to observe it 